In [1]:
# %pip install jovian

In [2]:
# import jovian

In [3]:
# jovian.commit(filename='data.ipynb', project='News Recommender Capstone')

In [4]:
import pandas as pd
import numpy as np

import plotly.express as px

In [5]:
# from shared_data import *
# data = SharedData()

In [6]:
txns = pd.read_csv('../data/consumer_transanctions.csv')
cnt = pd.read_csv('../data/platform_content.csv')

C:\Users\ksrs9\AppData\Local\Temp\ipykernel_24244\1419563702.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  txns = pd.read_csv('../data/consumer_transanctions.csv')


In [7]:
txns.head()

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,consumer_device_info,consumer_location,country
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,content_watched,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,content_watched,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,content_followed,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,content_watched,-7820640624231356730,-445337111692715325,561148 1178424124714,NaN,NaN,NaN


In [8]:
cnt.head()

,event_timestamp,interaction_type,item_id,producer_id,producer_session_id,producer_device_info,producer_location,producer_country,item_type,item_url,title,text_description,language
0,1459192779,content_pulled_out,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,content_present,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,content_present,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,content_present,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,content_present,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [9]:
# shared_data.get_data().store_transactions(txns)
# shared_data.get_data().store_content(cnt)

#### Data preparation

In [10]:
content = cnt

In [11]:
content.head()

,event_timestamp,interaction_type,item_id,producer_id,producer_session_id,producer_device_info,producer_location,producer_country,item_type,item_url,title,text_description,language
0,1459192779,content_pulled_out,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,content_present,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,content_present,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,content_present,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,content_present,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


Remove all docs that are not in English

In [12]:
content.language.value_counts()

en    2264
pt     850
la       4
es       2
ja       2
Name: language, dtype: int64

In [13]:
content.shape

(3122, 13)

In [14]:
content = content[content['language'] == 'en']

In [15]:
content.shape

(2264, 13)

Remove articles with duplicated entries

In [16]:
no_dups = content.sort_values('event_timestamp').drop_duplicates(subset=['title', 'text_description'], keep='last')

In [17]:
no_dups.head()

,event_timestamp,interaction_type,item_id,producer_id,producer_session_id,producer_device_info,producer_location,producer_country,item_type,item_url,title,text_description,language
1,1459193988,content_present,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,content_present,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,content_present,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,content_present,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,content_present,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en


In [18]:
no_dups.reset_index(inplace=True)

In [19]:
no_dups.interaction_type.value_counts()

content_present       2153
content_pulled_out      38
Name: interaction_type, dtype: int64

In [20]:
no_dups[no_dups['title'] == "Ethereum, a Virtual Currency, Enables Transactions That Rival Bitcoin's"]

,index,event_timestamp,interaction_type,item_id,producer_id,producer_session_id,producer_device_info,producer_location,producer_country,item_type,item_url,title,text_description,language
0,1,1459193988,content_present,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en


In [21]:
content[content['title'] == "Ethereum, a Virtual Currency, Enables Transactions That Rival Bitcoin's"]

,event_timestamp,interaction_type,item_id,producer_id,producer_session_id,producer_device_info,producer_location,producer_country,item_type,item_url,title,text_description,language
0,1459192779,content_pulled_out,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,content_present,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en


The entry in the no duplicates DataFrame is the one with the older timestamp. Makes sense.

In [22]:
cnt = no_dups

In [23]:
user_topics = []

Based on the user topics, the recommended articles change.

* At the start of the day, recommend 10 articles based on user interests
* Adjust user interests based on
    * Direct feedback
    * Clicks
    * Time spent on the article page (some threshold)
* When a new user is created, user will choose some topics
* New articles should be recommended based on collaborative and content-based filtering

### Topic modeling on articles

### Data preparation

Introduce a ratings column

In [24]:
def to_rating(val):
    if val == 'content_followed':
        return 5
    if val == 'content_commented_on':
        return 4
    if val == 'content_saved':
        return 3
    if val == 'content_liked':
        return 2
    return 1

In [25]:
txns.interaction_type.value_counts()

content_watched         61086
content_liked            5745
content_saved            2463
content_commented_on     1611
content_followed         1407
Name: interaction_type, dtype: int64

In [26]:
txns['rating'] = txns.interaction_type.apply(lambda x: to_rating(x))

In [27]:
txns.head()

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,consumer_device_info,consumer_location,country,rating
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1
1,1465412560,content_watched,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1
2,1465416190,content_watched,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1
3,1465413895,content_followed,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,5
4,1465412290,content_watched,-7820640624231356730,-445337111692715325,561148 1178424124714,NaN,NaN,NaN,1


#### Adjust IDs

In [28]:
class IdHelper:
    _map = {}
    _id = 1

    def translate(self, id):
        # If a mapping exists for id, then return the mapping
        # Otherwise, create a new mapping, store it, and return it
        if id in self._map:
            return self._map[id]
        new_id = self.__new_id__()
        self._map[id] = new_id
        return new_id

    def __new_id__(self):
        num = self._id
        self._id += 1
        return num

In [29]:
consumer_helper = IdHelper()
item_helper = IdHelper()

In [30]:
txns['consumer_id_adj'] = txns['consumer_id'].map(lambda x: consumer_helper.translate(x))

In [31]:
txns.head()

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,consumer_device_info,consumer_location,country,rating,consumer_id_adj
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1,1
1,1465412560,content_watched,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1,2
2,1465416190,content_watched,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1,3
3,1465413895,content_followed,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,5,4
4,1465412290,content_watched,-7820640624231356730,-445337111692715325,561148 1178424124714,NaN,NaN,NaN,1,5


In [32]:
txns['item_id_adj'] = txns['item_id'].map(lambda x: item_helper.translate(x))

In [33]:
txns.head()

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,consumer_device_info,consumer_location,country,rating,consumer_id_adj,item_id_adj
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1,1,1
1,1465412560,content_watched,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1,2,2
2,1465416190,content_watched,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1,3,3
3,1465413895,content_followed,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,5,4,3
4,1465412290,content_watched,-7820640624231356730,-445337111692715325,561148 1178424124714,NaN,NaN,NaN,1,5,4


In [34]:
cnt.head()

,index,event_timestamp,interaction_type,item_id,producer_id,producer_session_id,producer_device_info,producer_location,producer_country,item_type,item_url,title,text_description,language
0,1,1459193988,content_present,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,2,1459194146,content_present,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
2,3,1459194474,content_present,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
3,4,1459194497,content_present,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
4,5,1459194522,content_present,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en


In [35]:
cnt['item_id_adj'] = cnt['item_id'].map(lambda x: item_helper.translate(x))

In [36]:
cnt.head()

,index,event_timestamp,interaction_type,item_id,producer_id,producer_session_id,producer_device_info,producer_location,producer_country,item_type,item_url,title,text_description,language,item_id_adj
0,1,1459193988,content_present,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en,1190
1,2,1459194146,content_present,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en,811
2,3,1459194474,content_present,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en,559
3,4,1459194497,content_present,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en,2988
4,5,1459194522,content_present,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en,1191


### EDA

#### Checking for missing values

In [37]:
txns.isna().sum()

event_timestamp             0
interaction_type            0
item_id                     0
consumer_id                 0
consumer_session_id         0
consumer_device_info    15394
consumer_location       15405
country                 15394
rating                      0
consumer_id_adj             0
item_id_adj                 0
dtype: int64

In [38]:
txns.shape

(72312, 11)

In [39]:
cnt.isna().sum()

index                      0
event_timestamp            0
interaction_type           0
item_id                    0
producer_id                0
producer_session_id        0
producer_device_info    1727
producer_location       1727
producer_country        1727
item_type                  0
item_url                   0
title                      0
text_description           0
language                   0
item_id_adj                0
dtype: int64

In [40]:
cnt.shape

(2191, 15)

Drop the columns with missing values, since we won't use those in the analysis.

In [41]:
txns.drop(columns=['consumer_device_info', 'consumer_location', 'country'], inplace=True)

In [42]:
txns.shape

(72312, 8)

In [44]:
cnt.drop(columns=['producer_device_info', 'producer_location', 'producer_country'], inplace=True)

In [45]:
cnt.shape

(2191, 12)

#### Checking for duplicated ratings

In [46]:
txns.head()

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,rating,consumer_id_adj,item_id_adj
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,1,1,1
1,1465412560,content_watched,8890720798209849691,-1032019229384696495,3621737643587579081,1,2,2
2,1465416190,content_watched,310515487419366995,-1130272294246983140,2631864456530402479,1,3,3
3,1465413895,content_followed,310515487419366995,344280948527967603,-3167637573980064150,5,4,3
4,1465412290,content_watched,-7820640624231356730,-445337111692715325,561148 1178424124714,1,5,4


In [47]:
txns_2 = txns[['consumer_id_adj', 'item_id_adj', 'rating']]

In [48]:
txns_2.head()

,consumer_id_adj,item_id_adj,rating
0,1,1,1
1,2,2,1
2,3,3,1
3,4,3,5
4,5,4,1


In [49]:
duplicates = txns[txns.duplicated(subset=['consumer_id_adj', 'item_id_adj'], keep=False)]

In [50]:
duplicates.sort_values(by=['consumer_id_adj', 'item_id_adj', 'rating'], inplace=True)

C:\Users\ksrs9\AppData\Local\Temp\ipykernel_24244\4242022127.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates.sort_values(by=['consumer_id_adj', 'item_id_adj', 'rating'], inplace=True)


In [51]:
duplicates.head()

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,rating,consumer_id_adj,item_id_adj
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,1,1,1
34,1465413046,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,1,1,1
1647,1465481798,content_liked,8890720798209849691,-8845298781299428018,2199204395214244565,2,1,2
1651,1465481662,content_saved,8890720798209849691,-8845298781299428018,2199204395214244565,3,1,2
25568,1460648226,content_watched,5688279681867464747,-8845298781299428018,970398407890208999,1,1,28


There are duplicated entries i.e., the same user has interacted with the same article multiple times.

Since multiple interactions could mean that a user liked an article, the duplicates must be considered in the analysis.

#### For "duplicated" transactions, calculate the average rating of the user for that article

In [80]:
grp = duplicates.groupby(by=['consumer_id_adj', 'item_id_adj'])['rating'].mean()

In [81]:
grp.head()

consumer_id_adj  item_id_adj
1                1              1.000000
                 2              2.500000
                 28             1.250000
                 42             2.000000
                 68             2.142857
Name: rating, dtype: float64

In [82]:
grp_df = pd.DataFrame(grp)

In [83]:
grp_df.head()

rating
consumer_id_adj item_id_adj          
1               1            1.000000
                2            2.500000
                28           1.250000
                42           2.000000
                68           2.142857

Renaming the rating column to avoid any potential clash when merged with the original

In [84]:
grp_df.columns = ['rating_sum']

In [85]:
grp_df.head()

rating_sum
consumer_id_adj item_id_adj            
1               1              1.000000
                2              2.500000
                28             1.250000
                42             2.000000
                68             2.142857

In [86]:
grp_df.reset_index(inplace=True)

In [87]:
grp_df.head()

,consumer_id_adj,item_id_adj,rating_sum
0,1,1,1.000000
1,1,2,2.500000
2,1,28,1.250000
3,1,42,2.000000
4,1,68,2.142857


Check distributions of ratings

In [88]:
grp_df.describe(percentiles=[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0])

,consumer_id_adj,item_id_adj,rating_sum
count,16640.000000,16640.000000,16640.000000
mean,368.352043,1264.775120,1.344271
std,395.042294,886.443286,0.506015
min,1.000000,1.000000,1.000000
5%,7.000000,72.000000,1.000000
10%,21.000000,145.900000,1.000000
15%,32.850000,216.000000,1.000000
20%,53.000000,316.800000,1.000000
25%,69.000000,415.000000,1.000000
30%,85.000000,533.000000,1.000000


In [69]:
fig = px.box(grp_df, y='rating_sum')
fig.show()

A majority of the articles are rated 2 or lower. Only a very small number of transactions have a high rating. However, these are not outliers. This is expected, as users would only like a small percentage of the articles in the system.

#### Add the adjusted rating back to the original transactions DataFrame

In [89]:
no_dups = txns.drop_duplicates(subset=['consumer_id_adj', 'item_id_adj'])

In [90]:
no_dups.head()

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,rating,consumer_id_adj,item_id_adj
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,1,1,1
1,1465412560,content_watched,8890720798209849691,-1032019229384696495,3621737643587579081,1,2,2
2,1465416190,content_watched,310515487419366995,-1130272294246983140,2631864456530402479,1,3,3
3,1465413895,content_followed,310515487419366995,344280948527967603,-3167637573980064150,5,4,3
4,1465412290,content_watched,-7820640624231356730,-445337111692715325,561148 1178424124714,1,5,4


In [91]:
no_dups.sort_values(by=['consumer_id_adj', 'item_id_adj', 'rating'], inplace=True)

C:\Users\ksrs9\AppData\Local\Temp\ipykernel_24244\744968623.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [92]:
no_dups.head()

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,rating,consumer_id_adj,item_id_adj
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,1,1,1
1647,1465481798,content_liked,8890720798209849691,-8845298781299428018,2199204395214244565,2,1,2
13844,1462296634,content_watched,3460026829794173084,-8845298781299428018,-1096019825082524173,1,1,8
25568,1460648226,content_watched,5688279681867464747,-8845298781299428018,970398407890208999,1,1,28
41857,1470773847,content_watched,-5557505576293652420,-8845298781299428018,5449106049355767894,1,1,38


In [93]:
duplicates.shape

(48242, 8)

In [94]:
no_dups.shape

(40710, 8)

In [95]:
txns.shape

(72312, 8)

Merge the two DataFrames

In [96]:
txns_merged = pd.merge(left=no_dups, right=grp_df, left_on=['consumer_id_adj', 'item_id_adj'], right_on=['consumer_id_adj', 'item_id_adj'], how='left')

In [97]:
txns_merged.sort_values(by=['consumer_id_adj', 'item_id_adj', 'rating'], inplace=True)

In [98]:
txns_merged.head(25)

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,rating,consumer_id_adj,item_id_adj,rating_sum
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,1,1,1,1.000000
1,1465481798,content_liked,8890720798209849691,-8845298781299428018,2199204395214244565,2,1,2,2.500000
2,1462296634,content_watched,3460026829794173084,-8845298781299428018,-1096019825082524173,1,1,8,NaN
3,1460648226,content_watched,5688279681867464747,-8845298781299428018,970398407890208999,1,1,28,1.250000
4,1470773847,content_watched,-5557505576293652420,-8845298781299428018,5449106049355767894,1,1,38,NaN
5,1460648169,content_watched,-7681408188643141872,-8845298781299428018,970398407890208999,1,1,42,2.000000
6,1461867235,content_watched,-9192549002213406534,-8845298781299428018,-264113900783219266,1,1,52,NaN
7,1461867305,content_watched,5152069678055228801,-8845298781299428018,-264113900783219266,1,1,54,NaN
8,1466614562,content_watched,-4228415104574264137,-8845298781299428018,-5652127177064819661,1,1,68,2.142857
9,1464190235,content_watched,6437568358552101410,-8845298781299428018,-1157447994463607871,1,1,87,NaN


Rows that have rating_sum as NaN were not duplicated in the original. So, the summed rating would just be the rating for these rows.

In [99]:
txns_merged['ratings_merged'] = txns_merged.rating_sum.fillna(txns_merged.rating)

In [100]:
txns_merged.head(25)

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,rating,consumer_id_adj,item_id_adj,rating_sum,ratings_merged
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,1,1,1,1.000000,1.000000
1,1465481798,content_liked,8890720798209849691,-8845298781299428018,2199204395214244565,2,1,2,2.500000,2.500000
2,1462296634,content_watched,3460026829794173084,-8845298781299428018,-1096019825082524173,1,1,8,NaN,1.000000
3,1460648226,content_watched,5688279681867464747,-8845298781299428018,970398407890208999,1,1,28,1.250000,1.250000
4,1470773847,content_watched,-5557505576293652420,-8845298781299428018,5449106049355767894,1,1,38,NaN,1.000000
5,1460648169,content_watched,-7681408188643141872,-8845298781299428018,970398407890208999,1,1,42,2.000000,2.000000
6,1461867235,content_watched,-9192549002213406534,-8845298781299428018,-264113900783219266,1,1,52,NaN,1.000000
7,1461867305,content_watched,5152069678055228801,-8845298781299428018,-264113900783219266,1,1,54,NaN,1.000000
8,1466614562,content_watched,-4228415104574264137,-8845298781299428018,-5652127177064819661,1,1,68,2.142857,2.142857
9,1464190235,content_watched,6437568358552101410,-8845298781299428018,-1157447994463607871,1,1,87,NaN,1.000000


In [101]:
txns_merged.ratings_merged.describe()

count    40710.000000
mean         1.157177
std          0.397953
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          4.500000
Name: ratings_merged, dtype: float64

The rating is between 1 and 5, so that is good enough.

In [102]:
txns_merged.drop(columns=['rating_sum'], inplace=True)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
# scaler = MinMaxScaler(feature_range=(1, 5))
# x = scaler.fit_transform(txns_merged[['ratings_merged']]).reshape(-1)

In [ ]:
# scaler_2 = StandardScaler()
# y = scaler.fit_transform(txns_merged[['ratings_merged']]).reshape(-1)

In [ ]:
# txns_merged['ratings_scaled'] = x

In [ ]:
# txns_merged.head()

In [ ]:
# txns_merged.ratings_scaled.value_counts()

In [ ]:
# txns_merged.shape

In [103]:
txns_merged.head()

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,rating,consumer_id_adj,item_id_adj,ratings_merged
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,1,1,1,1.00
1,1465481798,content_liked,8890720798209849691,-8845298781299428018,2199204395214244565,2,1,2,2.50
2,1462296634,content_watched,3460026829794173084,-8845298781299428018,-1096019825082524173,1,1,8,1.00
3,1460648226,content_watched,5688279681867464747,-8845298781299428018,970398407890208999,1,1,28,1.25
4,1470773847,content_watched,-5557505576293652420,-8845298781299428018,5449106049355767894,1,1,38,1.00


In [104]:
txns_merged.rename(columns={'rating': 'rating_original'}, inplace=True)

In [105]:
txns_merged.head()

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,rating_original,consumer_id_adj,item_id_adj,ratings_merged
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,1,1,1,1.00
1,1465481798,content_liked,8890720798209849691,-8845298781299428018,2199204395214244565,2,1,2,2.50
2,1462296634,content_watched,3460026829794173084,-8845298781299428018,-1096019825082524173,1,1,8,1.00
3,1460648226,content_watched,5688279681867464747,-8845298781299428018,970398407890208999,1,1,28,1.25
4,1470773847,content_watched,-5557505576293652420,-8845298781299428018,5449106049355767894,1,1,38,1.00


In [106]:
# txns_merged.rename(columns={'ratings_scaled': 'rating'}, inplace=True)
txns_merged.rename(columns={'ratings_merged': 'rating'}, inplace=True)

In [107]:
txns_merged.head()

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,rating_original,consumer_id_adj,item_id_adj,rating
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,1,1,1,1.00
1,1465481798,content_liked,8890720798209849691,-8845298781299428018,2199204395214244565,2,1,2,2.50
2,1462296634,content_watched,3460026829794173084,-8845298781299428018,-1096019825082524173,1,1,8,1.00
3,1460648226,content_watched,5688279681867464747,-8845298781299428018,970398407890208999,1,1,28,1.25
4,1470773847,content_watched,-5557505576293652420,-8845298781299428018,5449106049355767894,1,1,38,1.00


In [108]:
txns = txns_merged

In [109]:
txns.describe()

,event_timestamp,item_id,consumer_id,rating_original,consumer_id_adj,item_id_adj,rating
count,4.071000e+04,4.071000e+04,4.071000e+04,40710.000000,40710.000000,40710.000000,40710.000000
mean,1.470525e+09,-5.840552e+16,-3.994571e+16,1.146917,430.560624,1493.917416,1.157177
std,7.535306e+06,5.375761e+18,5.107605e+18,0.539580,450.924680,913.586706,0.397953
min,1.457965e+09,-9.222795e+18,-9.223122e+18,1.000000,1.000000,1.000000,1.000000
25%,1.464379e+09,-4.762898e+18,-3.954277e+18,1.000000,81.000000,584.000000,1.000000
50%,1.469471e+09,-7.527881e+15,-8.938893e+16,1.000000,254.000000,1603.500000,1.000000
75%,1.475262e+09,4.555339e+18,3.829785e+18,1.000000,648.000000,2277.000000,1.000000
max,1.488310e+09,9.222265e+18,9.210531e+18,5.000000,1895.000000,2987.000000,4.500000


#### Plotting

In [110]:
px.histogram(txns, x='rating')

In [ ]:
cnt.head()

In [115]:
px.histogram(cnt, x='item_type')

In [116]:
px.histogram(cnt, x='language')

* When app is opened
    * Get top 10 recommendations for this user
        * If no data is available, pick top 10 from the chosen interests
        * Else, Get from the data
* When an article is read, AND one of the article recommendations is read, add it to transactions table
    * Infer the topics/genres of the article
    * Filtering should be done based on the topics/genres

Requirements
* Method to get topics/genres from content
* Generate user-article matrix
* Generate article-user matrix

### Topic Modelling using LDA

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

nltk.download('wordnet')
nltk.download('omw-1.4')

# import numpy as np
# np.random.seed(2018)

In [ ]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

def lemmatize_and_stem(text):
    return stemmer.stem(lemmatizer.lemmatize(text, pos='v'))

In [ ]:
def preprocess(text):
    result = []
    for x in simple_preprocess(text):
        if x not in STOPWORDS and len(x) > 3:
            result.append(lemmatize_and_stem(x))
    return result

In [ ]:
sample_desc = cnt.iloc[0].text_description

In [ ]:
preprocess(sample_desc)

In [ ]:
processed_docs = cnt['text_description'].map(preprocess)

In [ ]:
processed_docs[:10]

In [ ]:
dic = Dictionary(processed_docs)

In [ ]:
dic.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dic.doc2bow(x) for x in processed_docs]

In [ ]:
bow_corpus[0]

In [ ]:
bow_doc_0 = bow_corpus[0]

In [ ]:
print(f'Index: {bow_doc_0[0][0]}, Word: {dic[bow_doc_0[0][0]]}, Count: {bow_doc_0[0][1]}')

In [ ]:
from gensim import corpora, models

In [ ]:
tfidf = models.TfidfModel(bow_corpus)
bow_corpus_tfidf = tfidf[bow_corpus]

In [ ]:
from pprint import pprint
for x in bow_corpus_tfidf:
    pprint(x)
    break

In [ ]:
lda = models.LdaMulticore(bow_corpus, num_topics=10, id2word=dic, passes=2, workers=2)

In [ ]:
for idx, topic in lda.print_topics(-1):
    print(f'Topic: {idx}\n\tWords: {topic}\n')

In [ ]:
lda_tfidf = models.LdaMulticore(bow_corpus_tfidf, num_topics=10, id2word=dic, passes=2, workers=2)
for idx, topic in lda_tfidf.print_topics(-1):
    print(f'Topic: {idx}\n\tWords: {topic}\n')

## Getting articles for a User

Consider user-based collaborative filtering, and ALS. Whichever gives the best result would be the model to use.

#### User-based collaborative filtering

In [122]:
n_users = txns.consumer_id_adj.nunique()

In [123]:
n_articles = txns.item_id_adj.nunique()

In [119]:
txns.consumer_id.values

array([-8845298781299428018, -8845298781299428018, -8845298781299428018,
       ...,  8641138310821193516,  8641138310821193516,
        8113637998311268223], dtype=int64)

In [124]:
print(f'Num users: {n_users}, Num articles: {n_articles}')

Num users: 1895, Num articles: 2987


### Train test split

In [125]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(txns, test_size=0.3, random_state=42)

In [126]:
train.shape

(28497, 9)

In [127]:
test.shape

(12213, 9)

In [128]:
train.describe(percentiles=[0.25, 0.5, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0])

,event_timestamp,item_id,consumer_id,rating_original,consumer_id_adj,item_id_adj,rating
count,2.849700e+04,2.849700e+04,2.849700e+04,28497.000000,28497.000000,28497.000000,28497.000000
mean,1.470555e+09,-9.210028e+16,-5.301548e+16,1.147489,430.525038,1499.067516,1.156979
std,7.530892e+06,5.369607e+18,5.120330e+18,0.541093,450.984328,912.874316,0.400637
min,1.457965e+09,-9.222795e+18,-9.223122e+18,1.000000,1.000000,1.000000,1.000000
25%,1.464608e+09,-4.798432e+18,-3.978390e+18,1.000000,81.000000,591.000000,1.000000
50%,1.469539e+09,-4.212628e+16,-8.938893e+16,1.000000,254.000000,1614.000000,1.000000
75%,1.475262e+09,4.441012e+18,3.829785e+18,1.000000,643.000000,2277.000000,1.000000
80%,1.476981e+09,5.350830e+18,4.432373e+18,1.000000,761.000000,2425.800000,1.000000
85%,1.478886e+09,6.519443e+18,5.746645e+18,1.000000,928.000000,2547.000000,1.500000
90%,1.481900e+09,7.434271e+18,7.167125e+18,1.000000,1122.000000,2714.000000,1.500000


### User-Article matrix

Since this is collaborative filtering, we will consider the transactions matrix. From this, we construct a matrix of the ratings given by users for each product.

Populate the training matrix

In [129]:
def create_and_populate_user_article_matrix(data):
    data_matrix = np.zeros((n_users, n_articles))

    for line in data.itertuples():
        # print(line)
        # print(type(line))
        # print(f'UserId: {line.consumer_id_adj}, ArticleId: {line.item_id_adj}, Rating: {line.rating}')
        # break
        user_id = line.consumer_id_adj
        article_id = line.item_id_adj
        rating = line.rating

        data_matrix[user_id - 1, article_id - 1] = rating
    
    return data_matrix

Fill the training matrix with rating values

In [130]:
data_matrix = create_and_populate_user_article_matrix(train)

In [131]:
data_matrix

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.83333333, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [132]:
data_matrix.shape

(1895, 2987)

Dimensions match the number of unique users & articles

Populate the testing matrix

In [133]:
data_matrix_test = create_and_populate_user_article_matrix(test)

In [134]:
data_matrix_test

array([[0. , 2.5, 0. , ..., 0. , 0. , 0. ],
       [0. , 2. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 1.8, ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [135]:
data_matrix_test.shape

(1895, 2987)

Create matrix with full data

In [136]:
# data_matrix_full = create_and_populate_user_article_matrix(txns)

In [137]:
# data_matrix_full

In [138]:
# data_matrix_full.shape

### Pairwise Distance

In [139]:
from sklearn.metrics.pairwise import pairwise_distances

In [140]:
user_similarity = 1 - pairwise_distances(data_matrix, metric='cosine')

In [141]:
user_similarity

array([[1.        , 0.15926467, 0.04880552, ..., 0.        , 0.        ,
        0.        ],
       [0.15926467, 1.        , 0.08667542, ..., 0.        , 0.        ,
        0.        ],
       [0.04880552, 0.08667542, 1.        , ..., 0.        , 0.04386611,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.04386611, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [142]:
user_similarity.shape

(1895, 1895)

Take the transpose of the data matrix in order to calculate the article similarity.

In [143]:
# data_matrix.shape

In [144]:
# data_matrix.T.shape

In [145]:
article_similarity = 1 - pairwise_distances(data_matrix.T, metric='cosine')

In [146]:
article_similarity

array([[1.        , 0.24889563, 0.11910292, ..., 0.        , 0.        ,
        0.        ],
       [0.24889563, 1.        , 0.02202395, ..., 0.        , 0.        ,
        0.        ],
       [0.11910292, 0.02202395, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [147]:
article_similarity.shape

(2987, 2987)

### Get dot product of data matrix with similarity matrix

In [148]:
user_similarity.shape

(1895, 1895)

In [149]:
data_matrix_test.shape

(1895, 2987)

In [150]:
article_prediction = np.dot(user_similarity, data_matrix_test)

In [151]:
article_prediction.shape

(1895, 2987)

In [152]:
article_pred_df = pd.DataFrame(article_prediction)

In [153]:
article_pred_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2977,2978,2979,2980,2981,2982,2983,2984,2985,2986
0,0.143673,3.254799,0.245405,0.044782,0.022328,0.532970,0.570999,0.048806,0.202181,0.256083,...,0.0,0.004591,0.056674,0.0,0.000000,0.006644,0.021842,0.0,0.0,0.0
1,0.103099,3.744892,0.784183,0.166364,0.038610,0.863883,0.854128,0.086675,0.524641,1.027532,...,0.0,0.214345,0.203155,0.0,0.007302,0.047056,0.005316,0.0,0.0,0.0
2,0.021726,0.838452,2.226006,0.065983,0.049780,0.342300,0.186129,1.000000,0.376136,0.794697,...,0.0,0.118548,0.071983,0.0,0.000000,0.005184,0.000000,0.0,0.0,0.0
3,0.000000,0.365032,1.949186,0.000000,0.000000,0.148945,0.023038,0.000000,0.016471,0.454616,...,0.0,0.091701,0.000000,0.0,0.000000,0.036026,0.000000,0.0,0.0,0.0
4,0.019866,0.821098,0.426121,1.089345,0.039450,0.300026,0.190112,0.065983,0.283924,0.436737,...,0.0,0.238282,0.079827,0.0,0.053516,0.025284,0.023377,0.0,0.0,0.0


In [154]:
txns.consumer_id_adj.value_counts()

7       961
21      669
2       648
27      585
114     437
       ... 
456       1
1526      1
1518      1
1513      1
1895      1
Name: consumer_id_adj, Length: 1895, dtype: int64

#### Test for one user

In [155]:
test.head()

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,rating_original,consumer_id_adj,item_id_adj,rating
5712,1469717171,content_liked,-3780822597455574960,-7496361692498935601,-7184363612346079546,2,39,1750,1.500000
6946,1474569261,content_watched,9038543365726770177,3636910968448833585,-2350411385499638437,1,52,2273,1.000000
29727,1468410949,content_watched,4061646345741687197,1722265956540512475,2757554493783720116,1,607,1572,1.000000
21712,1464954966,content_watched,4277778562135834208,1493119272824026132,8271842215221662742,1,288,379,1.666667
33830,1471957935,content_watched,-8508606815368130888,1492042537097239970,-6546380720258601534,1,855,1993,1.000000


In [156]:
test_user_id = 288
test_user_idx = test_user_id - 1

In [159]:
test_user_id in test.consumer_id_adj.values

True

In [160]:
article_pred_df.iloc[test_user_idx]

0       0.072147
1       0.811299
2       0.458321
3       0.120408
4       0.126940
          ...   
2982    0.052005
2983    0.000000
2984    0.000000
2985    0.000000
2986    0.000000
Name: 287, Length: 2987, dtype: float64

In [194]:
article_recommendation = pd.DataFrame(article_pred_df.iloc[test_user_idx].sort_values(ascending=False))

In [195]:
article_recommendation

,287
2934,3.824871
2713,3.758974
2362,3.553203
202,3.551166
2373,3.340964
...,...
1125,0.000000
2403,0.000000
2404,0.000000
2406,0.000000


In [196]:
article_recommendation.reset_index(inplace=True)

In [197]:
article_recommendation.head()

,index,287
0,2934,3.824871
1,2713,3.758974
2,2362,3.553203
3,202,3.551166
4,2373,3.340964


Since the matrix is zero-based, the article ID index that we get is also zero-based. However, our IDs are one-based. So, convert the article ID to one-based by adding 1.

In [198]:
article_recommendation['index'] = article_recommendation['index'] + 1

In [199]:
article_recommendation.head()

,index,287
0,2935,3.824871
1,2714,3.758974
2,2363,3.553203
3,203,3.551166
4,2374,3.340964


In [200]:
article_recommendation.rename(columns={'index': 'article_id', test_user_idx: 'score'}, inplace=True)

In [201]:
article_recommendation.head()

,article_id,score
0,2935,3.824871
1,2714,3.758974
2,2363,3.553203
3,203,3.551166
4,2374,3.340964


Merging with the content dataframe to get the article title.

In [202]:
merged = pd.merge(article_recommendation, cnt, left_on='article_id', right_on='item_id_adj', how='left')

In [203]:
merged.columns

Index(['article_id', 'score', 'index', 'event_timestamp', 'interaction_type',
       'item_id', 'producer_id', 'producer_session_id', 'item_type',
       'item_url', 'title', 'text_description', 'language', 'item_id_adj'],
      dtype='object')

In [204]:
keep = ['article_id', 'score', 'title', 'interaction_type']

In [205]:
merged = merged.drop(columns=[col for col in merged if col not in keep])

In [206]:
merged.head(10)

,article_id,score,interaction_type,title
0,2935,3.824871,content_present,Don't document your code. Code your documentat...
1,2714,3.758974,content_present,"The Languages, Frameworks and Tools You Should..."
2,2363,3.553203,content_present,How it feels to learn Javascript in 2016
3,203,3.551166,NaN,NaN
4,2374,3.340964,content_present,The hardest thing in computer science
5,1897,3.257581,content_present,Do You Suffer From Deployment Anxiety? - DZone...
6,2769,3.000364,content_present,Most Interesting APIs in 2016: Cognitive Compu...
7,1807,2.983759,content_present,10 Modern Software Over-Engineering Mistakes
8,2338,2.955844,NaN,NaN
9,1736,2.926833,NaN,NaN


In [207]:
cnt[cnt['item_id_adj'] == 203]

,index,event_timestamp,interaction_type,item_id,producer_id,producer_session_id,item_type,item_url,title,text_description,language,item_id_adj


Some articles have title as NaN. This is because they do not exist in the content DataFrame, meaning they were pulled out of the system, or that data was somehow lost.

These entries can be used for analysis. However, they must not be included in any results.

In [208]:
merged.shape

(2988, 4)

In [209]:
merged = merged[~(merged['title'].isna())]

In [210]:
merged.shape

(2130, 4)

Of the remaining suggestions, some might have been pulled out of the system. Filter those out.

In [211]:
merged[merged['interaction_type'] == 'content_pulled_out']

,article_id,score,interaction_type,title
1238,1125,0.199390,content_pulled_out,Why It's Time to Take Google's PC Operating Sy...
1281,2417,0.189390,content_pulled_out,Real World Swift Performance
1547,1285,0.132034,content_pulled_out,Making Apache Spark Easier to Use in Java with...
1658,1386,0.117486,content_pulled_out,How Google sets goals: OKRs - GV Library
1700,292,0.111432,content_pulled_out,How Leaders Can Let Go Without Losing Control
1902,1388,0.076255,content_pulled_out,The Art of the OKR
1903,1382,0.076255,content_pulled_out,tins ::: Rick Klau's weblog
2138,2730,0.043467,content_pulled_out,"Which countries study which languages, and wha..."
2146,905,0.042106,content_pulled_out,"Google acquires Synergyse, an interactive trai..."
2150,627,0.041389,content_pulled_out,How Companies Are Using Machine Learning to Ge...


In [213]:
merged = merged[merged['interaction_type'] != 'content_pulled_out']

In [214]:
merged.shape

(2094, 4)

In [215]:
merged.head()

,article_id,score,interaction_type,title
0,2935,3.824871,content_present,Don't document your code. Code your documentat...
1,2714,3.758974,content_present,"The Languages, Frameworks and Tools You Should..."
2,2363,3.553203,content_present,How it feels to learn Javascript in 2016
4,2374,3.340964,content_present,The hardest thing in computer science
5,1897,3.257581,content_present,Do You Suffer From Deployment Anxiety? - DZone...


### Evaluate the predictions of the Collaborative User-based model

In [216]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from math import sqrt

In [217]:
data_matrix

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.83333333, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [218]:
data_matrix_test

array([[0. , 2.5, 0. , ..., 0. , 0. , 0. ],
       [0. , 2. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 1.8, ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [219]:
article_prediction

array([[0.14367345, 3.25479919, 0.24540477, ..., 0.        , 0.        ,
        0.        ],
       [0.10309915, 3.74489189, 0.78418255, ..., 0.        , 0.        ,
        0.        ],
       [0.0217256 , 0.8384518 , 2.22600589, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.08443689, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.10645397, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.19704777, 0.0614806 , ..., 0.        , 0.        ,
        0.        ]])

In [220]:
data_matrix_test_nz = data_matrix_test.nonzero()

In [221]:
prediction = article_prediction[data_matrix_test_nz]

In [222]:
ground_truth = data_matrix_test[data_matrix_test_nz]

#### Mean Absolute Error

In [223]:
mean_absolute_error(prediction, ground_truth)

0.5569011778965313

#### Root Mean Square Error

In [224]:
sqrt(mean_squared_error(prediction, ground_truth))

0.8216710492578613

#### Precision

In [225]:
num_pred = 10

In [226]:
predictions = merged['article_id'].values[:num_pred]

In [227]:
predictions

array([2935, 2714, 2363, 2374, 1897, 2769, 1807, 2932, 2476, 2229],
      dtype=int64)

In [228]:
txns.head()

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,rating_original,consumer_id_adj,item_id_adj,rating
0,1465413032,content_watched,-3499919498720038879,-8845298781299428018,1264196770339959068,1,1,1,1.00
1,1465481798,content_liked,8890720798209849691,-8845298781299428018,2199204395214244565,2,1,2,2.50
2,1462296634,content_watched,3460026829794173084,-8845298781299428018,-1096019825082524173,1,1,8,1.00
3,1460648226,content_watched,5688279681867464747,-8845298781299428018,970398407890208999,1,1,28,1.25
4,1470773847,content_watched,-5557505576293652420,-8845298781299428018,5449106049355767894,1,1,38,1.00


In [229]:
txns[txns['consumer_id_adj'] == test_user_id]

,event_timestamp,interaction_type,item_id,consumer_id,consumer_session_id,rating_original,consumer_id_adj,item_id_adj,rating
21698,1465230354,content_watched,8890720798209849691,1493119272824026132,918623708503057845,1,288,2,1.000000
21699,1465833446,content_watched,-8518096793350810174,1493119272824026132,-1724033337448041615,1,288,64,1.000000
21700,1465822372,content_watched,-3027055440570405664,1493119272824026132,-1724033337448041615,1,288,80,1.000000
21701,1465563061,content_liked,-5148591903395022444,1493119272824026132,260765541898877539,2,288,83,1.333333
21702,1476460755,content_watched,2555983212310147009,1493119272824026132,6104541708046809122,1,288,84,2.000000
...,...,...,...,...,...,...,...,...,...
21778,1487274477,content_watched,-8627051188605351707,1493119272824026132,3819855359090302634,1,288,2936,1.500000
21779,1487607173,content_watched,-6872546942144599345,1493119272824026132,896304817123427163,1,288,2943,1.000000
21780,1487858086,content_watched,-5953227649059336919,1493119272824026132,-787379361716753332,1,288,2956,1.000000
21781,1487789940,content_watched,-4132331404553626868,1493119272824026132,-3912922529950147668,1,288,2957,1.000000


In [230]:
avg = txns.rating.mean()

In [231]:
avg

1.1571769808846615

In [233]:
def get_articles_that_user_liked(user_id):
    # For this, we get all the articles that user has given a rating of more than the average rating
    user_interactions = txns[(txns['consumer_id_adj'] == user_id) & (txns['rating'] > avg)].sort_values(by='rating', ascending=False)

    return user_interactions['item_id_adj'].values

Since in the txns DataFrame, all IDs are 1-indexed, we can use the test user ID as it is.

In [234]:
user_interactions = txns[(txns['consumer_id_adj'] == test_user_id) & (txns['rating'] > avg)].sort_values(by='rating', ascending=False)

In [235]:
user_interactions.shape

(32, 9)

In [236]:
keep = ['consumer_id_adj', 'item_id_adj', 'rating', 'ratings_merged']

In [237]:
user_interactions.drop(columns=[col for col in user_interactions.columns if col not in keep], inplace=True)

In [238]:
user_interactions.head(num_pred)

,consumer_id_adj,item_id_adj,rating
21746,288,2133,3.333333
21732,288,1696,3.000000
21757,288,2393,2.000000
21739,288,1891,2.000000
21765,288,2714,2.000000
21766,288,2723,2.000000
21769,288,2769,2.000000
21756,288,2388,2.000000
21753,288,2338,2.000000
21774,288,2821,2.000000


In [246]:
top_user_interactions = user_interactions['item_id_adj'].values[:num_pred]

In [243]:
set(predictions)

{1807, 1897, 2229, 2363, 2374, 2476, 2714, 2769, 2932, 2935}

In [248]:
set(top_user_interactions)

{1696, 1891, 2133, 2338, 2388, 2393, 2714, 2723, 2769, 2821}

Get intersection of predictions and user interactions

In [249]:
set(predictions) & set(top_user_interactions)

{2714, 2769}

In [250]:
precision = len(set(predictions) & set(top_user_interactions)) / num_pred

In [251]:
precision

0.2

Recall

In [252]:
user_liked = get_articles_that_user_liked(test_user_id)

In [253]:
len(set(user_liked))

32

In [254]:
# Filter user_interactions by rating > avg
top_user_interactions

array([2133, 1696, 2393, 1891, 2714, 2723, 2769, 2388, 2338, 2821],
      dtype=int64)

In [ ]:
recall = len(set(predictions) & set(user_interact_ids)) / len(set(user_interact_ids))

In [ ]:
recall

#### For sum, Precision = 0.1, Recall: 0.023255813953488372

#### For mean, Precision = 0.1, Recall: 0.043478260869565216

Some articles are correctly identified.

However, there is a massive skew in the data.

### How do we deal with skew here?

## Getting articles matching another article

Consider item-based collaborative filtering and content-based filtering

### Content-based filtering

Derive keywords from the article text

In [ ]:
cnt.columns

In [ ]:
cnt.head()

In [ ]:
cnt.text_description

In [ ]:
text_array = cnt.text_description.to_numpy()

In [ ]:
# text_array

In [ ]:
cnt.shape

In [ ]:
len(text_array)

In [ ]:
words_list = []

for line in text_array:
    words = str(line).lower().split()
    words_list.append(words)

In [ ]:
len(words_list)

In [ ]:
words_list

#### Create Dictionary, Bag of Words, tfidf model & Similarity matrix

In [ ]:
%pip install gensim

In [ ]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.similarities import MatrixSimilarity

# create a dictionary from words list
dictionary = Dictionary(words_list) 

#create corpus where the corpus is a bag of words for each document
corpus = [dictionary.doc2bow(doc) for doc in words_list] 

#create tfidf model of the corpus
tfidf = TfidfModel(corpus) 

# Create the similarity matrix. This is the most important part where we get the similarities between the movies.
sims = MatrixSimilarity(tfidf[corpus], num_features=len(dictionary))


In [ ]:
def article_recommendation(content):
    #split the keywords
    keywords = content.split() 

    #set the list of words to query_doc
    query_doc = keywords 
    
    # get a bag of words from the query_doc
    query_doc_bow = dictionary.doc2bow(query_doc) 

    #convert the regular bag of words model to a tf-idf model
    query_doc_tfidf = tfidf[query_doc_bow] 

    # get similarity values between input movie and all other movies
    similarity_array = sims[query_doc_tfidf] 

    #Convert to a Series
    similarity_series = pd.Series(similarity_array.tolist()) 

    #get the most similar movies 
    # similarity_output = similarity_series.sort_values(ascending=False)
    similarity_output = similarity_series
    return similarity_output

In [ ]:
test = cnt.iloc[12]['text_description']

In [ ]:
test

In [ ]:
recs = article_recommendation(test)

In [ ]:
recs[:10]

In [ ]:
recs_df = pd.DataFrame(recs, columns=['Score'])

In [ ]:
recs_df

In [ ]:
recs_df.isna().sum()

In [ ]:
# recs_df['title'] = cnt['title']
# recs_df['text_description'] = cnt['text_description']

In [ ]:
# recs_df = cnt.join(recs_df)

In [ ]:
recs_df = cnt.join(recs_df)

In [ ]:
recs_df.head()

In [ ]:
recs_df.sort_values(by='Score', ascending=False)

In [ ]:
recs_df.isna().sum()

In [ ]:
keep = ['Score', 'title', 'text_description']

In [ ]:
recs_df.drop(columns=[col for col in recs_df if col not in keep], inplace=True)

In [ ]:
recs_df.head()

In [ ]:
recs_df.head(11)

In [ ]:
cnt.iloc[2186]

In [ ]:
txns.head()

## ALS

#### Create sparse User-Article matrix

In [ ]:
from scipy.sparse import csr_matrix

Random values in CSR matrix will be filled with alpha value

In [ ]:
txns.head()

In [ ]:
keep = ['consumer_id_adj', 'item_id_adj', 'rating']

In [ ]:
txns_mod = txns.drop(columns=[col for col in txns.columns if col not in keep])

In [ ]:
txns_mod.head()

In [ ]:
txns_mod.describe()

In [ ]:
dup = txns_mod[txns.duplicated(subset=['consumer_id_adj', 'item_id_adj'])]

In [ ]:
dup.head()

In [ ]:
dup.shape

In [ ]:
txns_mod.shape[0] - dup.shape[0]

In [ ]:
txns_mod[(txns_mod['consumer_id_adj'] == 3) & (txns_mod['item_id_adj'] == 3)]

In [ ]:
alpha = 40

In [ ]:
txns_mod.shape

In [ ]:
txns_mod.shape[0]

In [ ]:
x = [alpha] * txns_mod.shape[0]

In [ ]:
len(x)

In [ ]:
sparse_user_article = csr_matrix( ([alpha]*txns_mod.shape[0], (txns_mod['consumer_id_adj'], txns_mod['item_id_adj']) ))

In [ ]:
sparse_user_article

In [ ]:
n_users

In [ ]:
n_articles

Convert to array

In [ ]:
csr_user_array = sparse_user_article.toarray()

In [ ]:
csr_user_array

In [ ]:
txns[(txns['consumer_id_adj'] == 1) & (txns['item_id_adj'] == 1)]

In [ ]:
txns.consumer_id_adj.nunique()

In [ ]:
len(csr_user_array), len(csr_user_array[0])

Dimensions match with the matrix

In [ ]:
max(csr_user_array[1])

Create article-user sparse matrix

In [ ]:
sparse_article_user = sparse_user_article.T.tocsr()

In [ ]:
sparse_article_user

Shape matches

In [ ]:
csr_article_array = sparse_article_user.toarray()

#### Create train & test data

In [ ]:
%pip install implicit

In [ ]:
from implicit.evaluation import train_test_split

In [ ]:
sparse_article_user

In [ ]:
train, test = train_test_split(sparse_user_article, train_percentage=0.8)

In [ ]:
train

In [ ]:
test

#### Building the ALS Model

In [ ]:
from implicit.als import AlternatingLeastSquares

In [ ]:
model = AlternatingLeastSquares(factors=60, regularization=0.1, iterations=60, calculate_training_loss=False)

In [ ]:
# model

Training

In [ ]:
model.fit(train)

In [ ]:
# test

In [ ]:
test_user_id = 114

In [ ]:
user_interact_ids = get_articles_that_user_liked(test_user_id)

New Implicit API expects (user, item) sparse matrix as input

In [ ]:
model.recommend(test_user_id, sparse_user_article[test_user_id], N=20, filter_already_liked_items=False)

In [ ]:
ids, scores = model.recommend(test_user_id, sparse_user_article[test_user_id], N=20, filter_already_liked_items=False)

In [ ]:
out = pd.DataFrame({'article_id': ids, 'als_score': scores})

In [ ]:
# out

In [ ]:
out.head(10)

In [ ]:
# sparse_user_article

In [ ]:
# cons = list(txns.consumer_id_adj.values)

In [ ]:
# items = list(txns.item_id_adj.values)

In [ ]:
# 1647 in cons

In [ ]:
# 1647 in items

In [ ]:
# len(user_interact_ids)

In [ ]:
# set(out.article_id)

In [ ]:
set(out.article_id)

In [ ]:
set(user_interact_ids)

In [ ]:
set(user_interact_ids) & set(out.article_id)

In [ ]:
common = set(user_interact_ids) & set(out.article_id)

In [ ]:
out[out['article_id'].isin(common)]

### Evaluation is much better with ALS.

### Getting similar articles to a given article

#### Either Content-based filtering, or Collaborative Item-based filtering, or Hybrid model

In [ ]:
article_similarity.shape

In [ ]:
n_articles

In [ ]:
article_similarity

In [ ]:
article_simi_df = pd.DataFrame(article_similarity)

In [ ]:
article_simi_df.head()

In [ ]:
test_article_id = 5

In [ ]:
test_article_index = test_article_id - 1

In [ ]:
article_simi_df.iloc[test_article_index]

In [ ]:
article_simi_df.iloc[test_article_index].sort_values(ascending=False)

In [ ]:
article_recommendation = pd.DataFrame(article_simi_df.iloc[test_article_index].sort_values(ascending=False))

In [ ]:
article_recommendation.head()

In [ ]:
article_recommendation.reset_index(inplace=True)

In [ ]:
article_recommendation.head()

In [ ]:
article_recommendation.rename(columns={'index': 'article_id', test_article_index: 'similarity_score'}, inplace=True)

In [ ]:
article_recommendation.head()

Convert the IDs to 1-indexed

In [ ]:
# Convert article_recommendation.article_id from zero-based index to one-based index
article_recommendation['article_id'] = article_recommendation['article_id'] + 1

In [ ]:
article_recommendation.head()

In [ ]:
get_articles_that_user_liked(test_user_id)

In [ ]:
test_user_id

Filter out articles that the user has already read.

### Article-User Matrix

Same as User-Article matrix